# サイコロの簡単な例でPromiseを理解する

以下は実行する毎に1〜6を無作為に返し，かつコンソールに表示する関数. サイコロ．

In [22]:
function roll() {
    const n = Math.floor(Math.random() * 6 + 1);
    console.log(n);
    return n;
}

In [23]:
for (let i = 0; i < 10; i++) { roll(); }

3
1
6
1
6
6
1
2
1
5


5

上の例では一瞬でサイコロを10回投げたが、以下では、1秒ずつ間をおきながら投げるにはどうずればよいかという問題を例にして、JavaScriptでなぜPromiseが多用されるのかを理解していく．まずは(I)プロミスを用いない原始的な方法を理解し、次に(II)プロミスを用いると何がどのように改善されるのかを理解する．最後に(III)async/await構文によって、直感的に分かりやすく簡潔なコードになることを理解する．

## I. イベント駆動/コールバックによる方法

簡単な（一見、簡単に見える）問題からはじめる．

**[問題]** 1秒毎にサイコロを全部で3回投げるプログラムをつくる．

できれば以下のように書きたいのだが...．
```
1秒待つ;
roll();
1秒待つ;
roll();
1秒待つ;
roll();
```
JavaScriptでは，残念ながら「1秒待つ」のようなブロックする（何もしないで無駄に待つ）関数・命令はない．全体がシングルスレッドで動くからである．

実行のブロックが許されないJavaScriptでは、代わりにいわゆる**コールバック**（callback）のスタイルで**絶対にブロックしないようなコードを書いていく**ことになる．つまり`setTimeout()`という関数を用いて、1秒後に`roll()`を呼び出してくれるようにJavaScriptエンジンに登録する．`setTimeout()`関数**自身は登録をおこなうだけなので、ブロックされず、直ちに実行が終了**する（直ちに`hello`が表示される）ことに注意．登録してから（可能なら）およそ1秒後に`roll()`を呼び出すようなイベントが発生し`roll()`が実行される．これは例えばボタンをクリックすればイベントが発生し、あらかじめ登録しておいたイベントハンドラが実行されるのと同じ仕組みである．

In [25]:
setTimeout(roll, 3000);
console.log("hello");

hello


3


`setTimeout()`を用いた[問題]の解は以下のようになる．`setTimeout()`で登録した関数の中で`setTimeout()`を用いているので**3重に入れ子**になっている．一見しただけではどのように実行されるのか非常に分かりにくい．

In [26]:
setTimeout(function () {
    roll();
    setTimeout(function () {
        roll();
        setTimeout(function () {
            roll();
        }, 1000);
    }, 1000);
}, 1000);
console.log("hello");

hello


6
2
5


続いて先ほどよりやや難しい問題を考えてみる．

**[問題]** 1の目が出るまで1秒毎にサイコロを振るプログラムをつくる．

自然に思いつくのは、ループを用い、サイコロの目が1であれば脱出するというものである（以下）．しかし、このbreakは関数呼び出しをまたいでいるのでコンパイルエラーになる．

In [27]:
// 誤り
for (;;) {
    setTimeout(function () {
        if (roll() == 1) break;
    }, 1000);
}
console.log("hello");

SyntaxError: Illegal break statement

そこでbreakの代わりにフラグを使ってみるが、これも期待通り動かない．実際、このループの中身は一瞬で実行され、永久ループになる．この理由は`setTimeout()`が決してブロックしないことを考えれば当然である．

In [ ]:
// 誤り
// 実行すると永久ループになり結果が返ってこないのでJupyterのKernelを再起動すること
let cont = true;
while (cont) {
    setTimeout(function () {
        if (roll() == 1) cont = false;
    }, 1000);
}
console.log("hello");

同様に以下も誤り．

In [ ]:
// 誤り
// 実行すると永久ループになり結果が返ってこないのでJupyterのKernelを再起動すること
setTimeout(function () {
    while (roll() > 1) {
        setTimeout(function () {}, 1000);
    }
}, 1000);
console.log("hello");

ループで正しく実現するのは難しいので，関数呼び出しでループを表現したのが以下である．`setTimeout()`で登録される関数の中から再帰的に`roll_until_you_got_1()`を呼び出している．再び呼び出されるときは別のスタックで実行されるので、末尾再帰呼び出しでスタックオーバーフローする心配もない．

In [47]:
function roll_until_you_got_1() {
    setTimeout(function() {
        if (roll() > 1) {
            roll_until_you_got_1();
        }
    }, 1000);
}

roll_until_you_got_1();
console.log("hello");

hello


1


## II. プロミスによる方法

Javascript (ES 2015）の**プロミス**という用語は約束手形（promise）から来ている．すなわち後で手に入るかもしれない値を表すオブジェクトである．プロミスからその値を得ることを「プロミスを**解決**（resolve）する」と言う．プロミスを「売り手」と「買い手」をつなぐ「仲介者」あるいは「市場」のようなものだと想像するといいかもしれない．「売り手」が先に「市場」に出品していれば「買い手」は即、その商品（値）を得ることができるが、まだ出品していなければ待たされることになる．JavaScriptでは無駄に待つこと（ブロック）は厳禁なので、後日に商品が出品され次第、連絡をもらえるよう仲介者に頼んでおくことになる（コールバッグ関数の登録）．

プロミスを生成するときには、（まだ手に入っていない）値の代わりに、その値を「市場」に出品する「売り手」の動き方を与える．例えば、以下の関数`wait()`はmsミリ秒後に`undefined`値が手に入る（かもしれない）プロミスを返り値として返す．
```
function(resolve) {
    setTimeout(resolve, ms);
}
```
の部分が値`undefined`を出品する「売り手」の動きを記述した部分である．

In [6]:
function wait(ms) {
    return new Promise(function(resolve) {
        setTimeout(resolve, ms);
    })
}

「仲介者」に出品の連絡をくれるように依頼するのがプロミスの`then()`メソッドである．`then()`メソッドの引数として与えた関数（コールバック関数）が、あとで「売り手」が出品したときに呼び出される．

以下の例では`then()`の引数として与えたコールバック関数は、（可能なら）およそ3秒後に呼び出されることになる（手に入った値`undefined`は使わないのでここでは単に無視している）．`then()`**自身は直ちに完了する**のでhelloのほうが先に表示される．

In [31]:
wait(3000).then(function() {
    console.log("called"); // （可能なら）およそ3秒後にconsole.logが呼ばれる
});
console.log("hello");

hello


called


よって最初の問題の解は以下のように`then()`の連鎖で表せる．

In [32]:
wait(1000).then(function() {
    roll();
    return wait(1000).then(function() {
        roll();
        return wait(1000).then(function() {
            roll();
        });
    });
});
console.log("hello");

hello


6
2
2


このコードを見る限りでは，`setTimeout()`のときと実質的に同じで、プロミスを導入したメリットが無いようにみえる．実は コールバック関数もまたプロミスを返す関数で、`then()`メソッドもまたプロミスを返す．`then()`メソッドの返すプロミスを解決することは，そのコールバック関数の返すプロミスを解決することと同じである．このことをもっと正確に述べると，以下の等式が成り立つということである．

p.then(f).then(g) == p.then(function (x) { return f(x).then(g); })

この事実をを用いれば`then()`メソッドの入れ子は`then()`メソッドの連鎖（流れ作業）になる． このルールを当てはめて上記のコードを書き換えると以下のような入れ子のない`then()`の連鎖になる．

In [48]:
wait(1000).then(function() { roll(); return wait(1000); })
        　.then(function() { roll(); return wait(1000); })
        　.then(function() { roll(); });
console.log("hello");

hello


5
6
4


>**[関数型プログラミングになじみのある人向けのコメント]**
>
>`then()`に関する上記の規則は関数型プログラミングで**モナド法則**(monad law)と呼ばれるものの一種である．実際プロミスは一種の（最適化された）継続モナド（continuation monad）であり、`then()`メソッドはモナドのいわゆるbind演算に他ならない（注）．一般にモナドは、単純な計算を合成してより複雑な計算を実現する統一的な手段を提供する．プロミスの強みを一言で言い表すなら、プロミスはモナドの一種であり、それゆえに**単純な非同期計算を組み合わせてより複雑な非同期計算を実現する手段**を提供しているからだと言える．

>（注）もっとも，この解説では省略しているがプロミスには解決に失敗した時に呼び出されるコールバックもあるので，実際の話はもう少し複雑である．

以下は1の目がでるまで繰り返す問題のプロミスによる解である．

In [34]:
function roll_until_you_got_1() {
    wait(1000).then(function() {
        if (roll() > 1) {
            roll_until_you_got_1();
        }
    });
}

roll_until_you_got_1();
console.log("hello");

hello


6
3
2
3
3
6
1


上のコードは`setTimeout()`のときと実質的に同じに見え、このままではあまりうれしくない．だが、プロミスは他のデータと同じように変数に格納したり、引数として渡したり、返り値として返すことができる点が、`setTimeout()`のときとは異なる．この点を活かし、JavaScript (ES 2015)のgeneratorを用いれば、ループを使って書きなおすことができる．

この解説ではgeneratorについては説明しないが、以下の例：

In [23]:
function* code() {
    for (;;) {
        yield wait(1000);
        if (roll() == 1) break;
    }
    return 123;
}
var cont = code();

に対して

In [37]:
cont.next()

1


{ value: 123, done: true }

を1の目が返るまで繰り返し実行してみればだいたいの感じ（`code`が細切れに中断しながら実行されていく感じ）がつかめると思う．正確には`yield`のところで実行が毎回中断・再開される．この例では`yield`の引数がプロミス`wait(1000)`なので、value:として 毎回、このプロミスが返ってくる．done:の値が`true`になれば、`code`を最後まで実行し終えたということでありvalue:は返り値の123になる．

以下の`async()`は、generatorの`next()`メソッドを呼んで得られるプロミスを解決しながら繰り返し`next()`を呼びなおす関数である．

In [38]:
function async(code) {
    const cont = code();

    function await(a) {
        if (a.done) return;
        a.value.then(function() {
            await(cont.next());
        });
    }

    await(cont.next());
}

これを使えば、`wait(1000)`の呼び出しがあたかも1秒間ブロックするかのような感覚で（まるで同期的なコードを書くように）直感的に書いたコードを期待通りに実行できる．

In [39]:
async(function*() {
    for (;;) {
        yield wait(1000);
        if (roll() == 1) break;
    }
});

console.log("hello");

hello


1


## III. async/await構文の利用

このように`async()`, `await()`関数を用いれば、プロミスを用いるコードを直感的に分かりやすく記述できる．そこでJavaScript（ES 2016)ではこれを構文として取り入れている（つまり上記のような関数`async()`を自分で書かなくてよい）．

In [42]:
(async function() {
  await wait(3000);
  console.log("hello");
})()

Promise [Promise] {}

hello


await構文を用いれば、最終的に２つの問題の解は以下のように非常に直感的に簡単に与えることができる．

In [43]:
(async function() {
  await wait(1000);
  roll();
  await wait(1000);
  roll();
  await wait(1000);
  roll();
  console.log("hello");
})()

Promise [Promise] {}

3
4
3
hello


In [44]:
(async function() {
  for (;;) {
      await wait(1000);
      if (roll() == 1) break;
  }
  console.log("hello");
})()

Promise [Promise] {}

5
3
1
hello


`await`を関数の中で使うときは以下のように関数に`async`をつける．関数には返り値があり、これを無視せずに扱うためである．

In [45]:
async function roll_until_you_got_1() {
    for (;;) {
        await wait(1000);
        if (roll() == 1) break;
    }
}

roll_until_you_got_1();
console.log("hello");

hello


2
4
3
5
5
2
6
1


`async`のついた関数は返り値をプロミスとして返す．

In [46]:
async function f() {
    console.log("a");
    await wait(1000);
    console.log("b");
    return 123;
}

In [51]:
f()

a


Promise [Promise] {}

b


In [50]:
(async function() {
    await f();
})()

a


Promise [Promise] {}

b


## 練習問題

関数roll(ms)はmsミリ秒待ってからサイコロを投げる関数である．これを用いて上の二つの問題の解をあたえよ．

In [54]:
function roll(ms) {
    return new Promise(function (resolve) {
        setTimeout(function () {
            resolve(Math.floor(Math.random() * 6 + 1));
        }, ms);
    });
}

In [55]:
(async function() {
    await roll(1000);
})()

Promise [Promise] {}

**（参考）async(), await()関数のより完全な定義**

上で与えたasync(), await()はサイコロの例に即して簡略化されている．より汎用的で完全な定義は以下のようになる．

In [ ]:
function async(code) {
    return function(...args) {
        const cont = code.apply(this, args);

        function await({done, value}) {
            if (done) return Promise.resolve(value);
            return Promise.resolve(value).then(
                function (fullfilled) { return await(cont.next(fullfilled)); },
                function (rejected)   { return await(cont.throw(rejected));  }
            );
        }

        try {
            return await(cont.next());
        }
        catch (e) {
            return Promise.reject(e);
        }
    }
}